In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from openai import OpenAI
import requests
import re
from defines import coordinates_dict
import glob
import difflib
import pickle
from multiprocessing import Pool, cpu_count
import unicodedata
from ast import literal_eval
from statistics import mode
import os
from pathlib import Path
import shutil
from odf import text, teletype
from odf.opendocument import load

In [2]:
directory_path  = "./data/cleaned_data/**/"  # Replace with the path to your folder
all_csv_files_path  = glob.glob(f"{directory_path}/*.csv", recursive=True)

# Display the list of CSV files
print(f"CSV files in the folder:{len(all_csv_files_path)}")
for csv_file_path in all_csv_files_path: #sorted(all_csv_files_path, key=lambda x: Path(x).stem):
    print(csv_file_path)

CSV files in the folder:121
./data/cleaned_data/CSV_05_Dec/319.csv
./data/cleaned_data/CSV_05_Dec/311.csv
./data/cleaned_data/CSV_05_Dec/306.csv
./data/cleaned_data/CSV_05_Dec/339.csv
./data/cleaned_data/600_699/610.csv
./data/cleaned_data/600_699/617.csv
./data/cleaned_data/600_699/616.csv
./data/cleaned_data/600_699/621.csv
./data/cleaned_data/600_699/625.csv
./data/cleaned_data/600_699/615.csv
./data/cleaned_data/600_699/618.csv
./data/cleaned_data/600_699/614.csv
./data/cleaned_data/600_699/624.csv
./data/cleaned_data/600_699/619.csv
./data/cleaned_data/600_699/622.csv
./data/cleaned_data/600_699/613.csv
./data/cleaned_data/600_699/623.csv
./data/cleaned_data/600_699/627.csv
./data/cleaned_data/600_699/612.csv
./data/cleaned_data/600_699/620.csv
./data/cleaned_data/600_699/609.csv
./data/cleaned_data/CSV_06_July/151.csv
./data/cleaned_data/CSV_06_July/812.csv
./data/cleaned_data/CSV_06_July/838.csv
./data/cleaned_data/CSV_06_July/999.csv
./data/cleaned_data/CSV_06_July/14.csv
./dat

In [3]:
# drop columns
drop_column_list = ['scientificName', 'scientificName_matchingScore_kingdom_CloseGbiflist', \
               'scientificName_matchingScore_kingdom_CloseGbiflistLength',\
               'scientificName_matchingScore_kingdom_CloseGbif_Candidate', 'location_index', 
               'coordinates']

rename_columns = {'scientificNameGbif':'scientificName', 'location':'verbatimLocality', 'kingdomGbif': 'kingdom',
                  'latitude':'decimalLatitude', 'longitude':'decimalLongitude', 'locality':'verbatimLocality'}


matchingScore = 50.0

remove_rows = ['scientificName', 'eventDate']


def create_directory_from_file_path(file_path):
    # Extract the directory path
    directory_path = os.path.dirname(file_path)

    # Check if the directory exists
    if not os.path.exists(directory_path):
        # Create the directory
        os.makedirs(directory_path)
    return directory_path


def final_cleaning(csv_file_path):
    try:
        df = pd.read_csv(filepath_or_buffer=csv_file_path, encoding='utf-8')
        
        # drop columns
        df = df.drop(drop_column_list, axis=1, errors='ignore')

        # rename columns
        df = df.rename(columns=rename_columns)

        # remove rows
        df = df[df['scientificName'].notna()]
        df = df[df['eventDate'].notna()]

        # Filter rows based of matchingScore
        df = df[df['matchingScoreGbif']>0.5]

        # drop matching score columns
        df = df.drop(columns=['matchingScoreGbif'], axis=1, errors='ignore')

        # save csv data
        new_csv_file_path = csv_file_path.replace('./data/cleaned_data/', './data/final_data/')
        csv_file_path_dir = create_directory_from_file_path(new_csv_file_path)
        df.to_csv(new_csv_file_path, encoding='utf-8', index=False)
        
    except Exception as e:
        print(f"Error final cleaning data '{csv_file_path}': {e}") 
    
for csv_files in all_csv_files_path:
    final_cleaning(csv_files)

In [4]:
## clean organism quantity

def clean_organism_quantity(value):
    try:
        value1 = str(value).strip().replace(',', '.').replace('\s+', '')
        value1 = re.sub(r'[^0-9-+—‒.]', '', str(value1))
        if (float(value1) or int(value1)):
           return float(value1)
    except Exception as e:
        if re.match(r'[0-9-+—‒]', value1):
            return value1
        else:
            return None
            # print(value, e)
    return None

def final_cleaning(csv_file_path):
    try:
        df = pd.read_csv(filepath_or_buffer=csv_file_path, encoding='utf-8')
        
        df['organismQuantity'] = df['organismQuantity'].apply(clean_organism_quantity)

        # save csv data
        new_csv_file_path = csv_file_path.replace('./data/final_data/', './data/final_final_data/')
        csv_file_path_dir = create_directory_from_file_path(new_csv_file_path)
        df.to_csv(new_csv_file_path, encoding='utf-8', index=False)
        
    except Exception as e:
        print(f"Error final cleaning data '{csv_file_path}': {e}") 
    

directory_path  = "./data/final_data/**/"  # Replace with the path to your folder
all_csv_files_path  = glob.glob(f"{directory_path}/*.csv", recursive=True)

# Display the list of CSV files
print(f"CSV files in the folder:{len(all_csv_files_path)}")
for csv_file_path in all_csv_files_path: #sorted(all_csv_files_path, key=lambda x: Path(x).stem):
    print(csv_file_path)

for csv_files in all_csv_files_path:
    final_cleaning(csv_files)

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_15149/2835192249.py:5: SyntaxWarning: invalid escape sequence '\s'
  value1 = str(value).strip().replace(',', '.').replace('\s+', '')


CSV files in the folder:121
./data/final_data/CSV_05_Dec/319.csv
./data/final_data/CSV_05_Dec/311.csv
./data/final_data/CSV_05_Dec/306.csv
./data/final_data/CSV_05_Dec/339.csv
./data/final_data/600_699/610.csv
./data/final_data/600_699/617.csv
./data/final_data/600_699/616.csv
./data/final_data/600_699/621.csv
./data/final_data/600_699/625.csv
./data/final_data/600_699/615.csv
./data/final_data/600_699/618.csv
./data/final_data/600_699/614.csv
./data/final_data/600_699/624.csv
./data/final_data/600_699/619.csv
./data/final_data/600_699/622.csv
./data/final_data/600_699/613.csv
./data/final_data/600_699/623.csv
./data/final_data/600_699/627.csv
./data/final_data/600_699/612.csv
./data/final_data/600_699/620.csv
./data/final_data/600_699/609.csv
./data/final_data/CSV_06_July/151.csv
./data/final_data/CSV_06_July/812.csv
./data/final_data/CSV_06_July/838.csv
./data/final_data/CSV_06_July/999.csv
./data/final_data/CSV_06_July/14.csv
./data/final_data/CSV_06_July/47.csv
./data/final_data/CS